In [26]:
from helpers import *
from time import time
import pandas as pd
from tqdm import tqdm

In [2]:
point_set_df = pd.read_csv('./points_df_025.csv', header=0, index_col=0)
point_set_df

,Latitude,Longitude
0,51.737184,-0.620643
1,51.737184,-0.617012
2,51.737184,-0.613382
3,51.737183,-0.609751
4,51.737183,-0.606120
...,...,...
58243,51.238843,0.312049
58244,51.238815,0.315640
58245,51.238786,0.319231
58246,51.238757,0.322822


In [3]:
sample_point_set_df = point_set_df.sample(10)

In [4]:
API_key = os.getenv('OS_DATA_HUB_API_KEY_0')

In [5]:

land_type_list = []
for i in range(len(sample_point_set_df)):
  land_type = get_land_type(sample_point_set_df.iloc[i]['Latitude'], sample_point_set_df.iloc[i]['Longitude'], 0.25, API_key)
  land_type_list.append(land_type)

land_type_list

['Water,Building,Green_Space',
 'Water,Building,Green_Space',
 'Building,Green_Space,Urban_Area',
 'Green_Space',
 'Green_Space,Urban_Area',
 'Water,Building,Green_Space,Urban_Area',
 'Building,Green_Space',
 '',
 'Building,Green_Space,Urban_Area',
 '']

In [6]:
# Heathrow test
51.467911,-0.465889
land_type = get_land_type(51.467911, -0.465889, 10, API_key)
land_type

'Airport,Water,Building,Railway_Station,Green_Space,Urban_Area'

['Building',
 'Water',
 'Building',
 'Building',
 None,
 'Building',
 'Building',
 'Water',
 'Building',
 'Building']

In [26]:
def is_building_test(bbox):
    result_local = get_feature_type_in_bbox(bbox, 'Zoomstack_LocalBuildings')
    result_district = get_feature_type_in_bbox(bbox, 'Zoomstack_DistrictBuildings')
    print(len(result_local['features']))
    print(len(result_district['features']))
    print(result_local)
    if len(result_local['features']) > 0:
        return True
    elif len(result_district['features']) > 0:
        return True
    else:
        return False
    
is_building_test('51.503769,-0.130047,51.505492,-0.126742')

403 Client Error: Forbidden for url: https://api.os.uk/features/v1/wfs?service=wfs&key=SYftl2NvP35BS3NTa6MIuwv5Jo0meebK&request=GetFeature&version=2.0.0&typeNames=Zoomstack_LocalBuildings%3A&typeNames=Zoomstack_DistrictBuildings&outputFormat=GEOJSON&bbox=51.503769%2C-0.130047%2C51.505492%2C-0.126742


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Simple solution

- 1 api key
- Make 6 calls
- Wait a minute
- Repeat

In [52]:

def get_land_types_for_points_in_csv(csv_path, save_path, start_point_index, end_point_index, diameter_resolution, API_key):
  # Open CSV
  points_df = pd.read_csv(csv_path)
  # Get correct set of points
  subset_points_df = points_df.loc[start_point_index:end_point_index]
  
  print('Number of points to be processed:', len(subset_points_df))
  print('Start/End index (inclusive):', start_point_index, end_point_index)
  print('Start point:', subset_points_df.iloc[0]['Latitude'], ',', points_df.iloc[0]['Longitude'])
  print('End point:', points_df.iloc[-1]['Latitude'], ',', points_df.iloc[-1]['Longitude'])
  
  
  land_type_list = []
  # Get land type of those set 6 points
  t0 = time()
  for i in tqdm(range(len(subset_points_df))):
    if i % 10 == 0 and i != 0:
      print('Saving and waiting...')
      # Create or append to a csv while waiting
      df = pd.DataFrame(dtype='object', index=subset_points_df.index[:len(land_type_list)])
      df['Land_Type'] = land_type_list
      df.to_csv(save_path)
      print('Last batch of points:' , land_type_list)
      # Wait 1 minute to repeat
      while(time() - t0 < 2):
        continue
      t0 = time() # reset the timer
    
    land_type = get_land_type(subset_points_df.iloc[i]['Latitude'],
                              subset_points_df.iloc[i]['Longitude'],
                              diameter_resolution,
                              API_key)
    land_type_list.append(land_type)
  
  print('Saving...')
  df = pd.DataFrame(dtype='object', index=subset_points_df.index)
  df['Land_Type'] = land_type_list
  df.to_csv(save_path)
  print('Done.')



get_land_types_for_points_in_csv('./points_df_025.csv', './land_types.csv', 0, 100, 0.25, API_key)
pd.read_csv('./land_types.csv', header=0, index_col=0)

Number of points to be processed: 101
Start/End index (inclusive): 0 100
Start point: 51.73718391144484 , -0.6206426520917254
End point: 51.238728303226935 , 0.3264124829835313


  0%|          | 0/101 [00:00<?, ?it/s]

## More complicated Solution

- Maintain a dictionary of keys and time of last use
- Grab a usable key from the dictionary each time

In [46]:
pd.Series([0, 1, 2, 3, 4, 5, 6]).iloc[2:].index[:5]

RangeIndex(start=2, stop=7, step=1)

In [47]:
[1, 2, 3, 4, 5][:3]

[1, 2, 3]